In [1]:
import requests
import os
from dotenv import load_dotenv
load_dotenv()
import os
from supabase import create_client
import pandas as pd
import datetime
import logging
import numpy as np
from bs4 import BeautifulSoup

In [2]:
url_supabase = os.environ.get("SUPABASE_URL")
key = os.environ.get("SUPABASE_KEY")
supabase = create_client(url_supabase, key)

In [3]:
data_code = supabase.table("idx_company_profile").select("*").execute()
data = pd.DataFrame(data_code.data)

# Get Investing Links

In [4]:
symbols = [ticker.lower().replace(".jk", "") for ticker in data["symbol"][data["current_source"] == 2].tolist()]
avail_data =  [ 'bbca', 'amar', 'maba', 'cowl', 'btps', 'agrs', 'agro', 'life','bmas', 'bvic', 'mega', 'bsim', 'arto', 
                'mrei', 'asrm', 'bmri', 'bbri', 'home', 'lmpi', 'kmds', 'bpfi', 'smil', 'lpgi', 'bbkp', 'bris', 'bina', 
                'inet', 'bank', 'krah', 'dnar', 'amag', 'bcic', 'dcii', 'hill', 'plas', 'beks', 'hatm', 'pnbs', 'bbhi', 
                'nips', 'irsx', 'mcor', 'bbtn', 'mtwi', 'sstm', 'bgtg', 'maya', 'bhat', 'nisp', 'nobu', 'goll', 'bnga', 
                'imas', 'pnbn', 'bswd', 'pnin', 'bbyb', 'bjtm', 'babp', 'bbmd', 'abda', 'admf', 'kbri', 'jsky', 'baca', 
                'sdra', 'miti', 'tram', 'buah', 'btpn', 'bksw', 'bnba', 'bbsi', 'cuan', 'bnli', 'gsmf', 'asdm', 'casa', 
                'bdmn', 'pnlf', 'nusa', 'beef', 'skyb', 'sugi', 'smma', 'asmi', 'tugu', 'myrx', 'bbni', 'inpc', 'bnii', 
                'bjbr', 'hotl', 'army', 'duck', 'magp', 'npgf', 'lcgp', 'tril', 'forz']
na = [symbol for symbol in symbols if symbol not in avail_data]
na_list = [miss.upper() + ".JK" for miss in na]

In [9]:
data = data[data["symbol"].isin(na_list)]
companies = [company.replace("pt ", "").replace(" tbk", "").replace(" ", "-").replace(".", "").strip() for company in data["company_name"].str.lower()]
symbols = data["symbol"].tolist()

In [19]:
dict_na = {
    "symbols" : [],
    "href" : []
}
for stock in na:
    link = f"https://www.investing.com/search/?q={stock}"
    content = requests.get(link).text
    soup = BeautifulSoup(content, 'html.parser')
    search_results = soup.find_all(class_ = "js-inner-all-results-quote-item row")
    for search_result in search_results:
        text = search_result.get_text()
        if "Stock - Jakarta" in text:
            ticker = search_result.find("span", class_ = "second")
            if ticker.get_text().lower() == stock:
                dict_na["symbols"].append(stock)
                dict_na["href"].append(search_result["href"])

In [32]:
data_link = pd.DataFrame(dict_na)
data_link = data_link[data_link["symbols"] != "asha"]
leftover = {
    "unit" : "/equities/nusantara-inti",
    "trus" : "/equities/trust-finance-indonesia-tbk-pt",
    "gold" : "/equities/golden-retail",
    "euro" : "/equities/estee-gold-feet-pt-tbk",
    "digi" : "/equities/arkadia-digital",
    "food"  : "/equities/sentra-food",
    "glob" : "/equities/global-telesho",
    "real" : "/equities/repower-asia-indonesia-pt",
    "asha" : "/equities/cilacap-samudera-fishing-industry",
    "cntx" : "/equities/centex-tbk-sp"
}
lft = {
    "symbols" : [],
    "href" : []
}
for key, value in zip(leftover.keys(), leftover.values()):
    lft["symbols"].append(key)
    lft["href"].append(value)
lft = pd.DataFrame(lft)
data_link = pd.concat([data_link, lft])

In [46]:
ids = []
for link in data_link["href"]:
    try:
        url = f"https://www.investing.com{link}-financial-summary"
        html_content = requests.get(url).text
        soup = BeautifulSoup(html_content, "html.parser")
        check = soup.find(class_="headBtnWrapper float_lang_base_2 js-add-alert-widget")
        id = check["data-pair-id"]
        ids.append(id)
    except Exception as e:
        print(f"error pada link {link} dengan error: {e}")
        ids.append(np.NAN)

In [51]:
data_link = data_link.assign(id = ids)
data_link.to_csv("investing_link.csv", index = False)

In [34]:
status_dict = {
    "symbol" : [],
    "status" : [],
    "should_be_gross_margin" : []
}
failure = []
for symbol,company in zip(data_link["symbols"], data_link["href"]):
        link = f"https://www.investing.com{company}-financial-summary"
        status = str(requests.get(link))
        try:
            html_content = requests.get(link).text
            soup = BeautifulSoup(html_content, 'html.parser')
            values = soup.find("span", class_="float_lang_base_1")
            status_dict["should_be_gross_margin"].append(values.get_text())
            status_dict["symbol"].append(symbol.upper() + ".JK")
            status_dict["status"].append(status)
        except Exception as e:
            print(f"symbol {symbol} error: {e}")
            failure.append(symbol)

symbol rmko error: 'NoneType' object has no attribute 'get_text'
symbol relf error: 'NoneType' object has no attribute 'get_text'


In [38]:
status_df = pd.DataFrame(status_dict)
[symbol for symbol in na_list if symbol not in status_df["symbol"][status_df["should_be_gross_margin"] == "Gross margin"].tolist()]

['KLAS.JK', 'RMKO.JK', 'RELF.JK']

3 symbol di atas tidak bisa discraping pada investing.com

# Scraper Quarterly

In [4]:
data_link = pd.read_csv("investing_link.csv")

In [5]:
data_link

,symbols,href,id
0,cbre,/equities/cakra-buana-resources-energi-pt-tbk,1198683
1,laju,/equities/pt-jasa-berdikari-logistics-tbk,1199667
2,atap,/equities/trimitra-prawara-goldland-tbk-pt,1168052
3,cash,/equities/cashlez-worldwide-indonesia-tbk,1161015
4,cbpe,/equities/pt-citra-buana-prasida-tbk,1198668
...,...,...,...
228,food,/equities/sentra-food,1117859
229,glob,/equities/global-telesho,101415
230,real,/equities/repower-asia-indonesia-pt,1155912
231,asha,/equities/cilacap-samudera-fishing-industry,1191905


In [6]:
expected_value = [
    # Income Statement
    "Total Revenue", "Net Income", "Diluted Weighted Average Shares",
    "Gross Profit", "Net Income Before Taxes", "Provision for Income Taxes",
    "Interest Expense (Income) - Net Operating", "Operating Income",
    "Cash & Due from Banks",

    # Balance Sheet
    "Total Assets", "Total Liabilities", "Total Current Liabilities", 
    "Total Equity", "Current Port. of LT Debt/Capital Leases", 
    "Total Long Term Debt", "Total Liabilities & Shareholders' Equity", 
    "Minority Interest", "Cash and Short Term Investments", 
    "Cash & Equivalents", "Total Current Assets",

    # Cash FLow
    "Cash From Operating Activities", "Free Cash Flow"
]

In [7]:
url_currency = 'https://raw.githubusercontent.com/supertypeai/sectors_get_conversion_rate/master/conversion_rate.json'
response = requests.get(url_currency)
data = response.json()
rate = float(data['USD']['IDR'])

In [8]:
data_list = []

def get_values(soup, data_dict):
    n_values = len(data_dict["year"])
    info = soup.find(class_="arial_11 lightgrayFont bottomRemark").get_text()
    currency = "IDR" if "IDR" in info else "USD"
    fold = "Millions" if "Millions" in info else "Billions"
    tbodies = soup.find("tbody")
    rows = tbodies.find_all("tr")
    for row in rows:
        td = row.find_all("td")
        key = td[0].find("span").get_text()
        values = [t.get_text() if t.get_text() != "-" else np.nan for t in td[1:]]
        if len(values) > 4:
            continue
        try:
            if currency == "USD":
                if fold == "Billions":
                    values = [float(value) * rate * 1000000000 for value in values]
                else:
                    values = [float(value) * rate * 1000000 for value in values]
            else:
                if fold == "Billions":
                    values = [float(value) * 1000000000 for value in values]
                else:
                    values = [float(value) * 1000000 for value in values]
        except Exception as e:
            print(f"error di {symbol}, dengan error: {e}")
        # print("values: ", values)
        if key in expected_value:
            data_dict[key] = values

for symbol, link in zip(data_link["symbols"].tolist()[:5], data_link["href"].tolist()[:5]):
    # Income Statement
    is_url = f"https://www.investing.com{link}-income-statement"
    html_content = requests.get(is_url).text
    soup = BeautifulSoup(html_content, "html.parser")
    period = soup.find("tr", class_ = "alignBottom")
    quarters = period.find_all("th")
    data_dict = {"symbols" : [], "year" : []}
    for quarter in quarters:
        try:
            year = quarter.find("span").get_text()
            day, month = quarter.find("div").get_text().split("/")
            date = str(year) + "-" + str(month) + "-" + str(day)
            data_dict["symbols"].append(symbol.upper() + ".JK")
            data_dict["year"].append(date)
        except:
            pass
    get_values(soup, data_dict)
    # Balance Sheet
    bs_url = f"https://www.investing.com{link}-balance-sheet"
    html_content = requests.get(bs_url).text
    soup = BeautifulSoup(html_content, "html.parser")
    get_values(soup, data_dict)
    # Cash Flow
    bs_url = f"https://www.investing.com{link}-cash-flow"
    html_content = requests.get(bs_url).text
    soup = BeautifulSoup(html_content, "html.parser")
    get_values(soup, data_dict)
    not_avail = [na for na in expected_value if na not in data_dict.keys()]
    for na in not_avail:
        data_dict[na] = [np.nan, np.nan, np.nan, np.nan]
    data_list.append(pd.DataFrame(data_dict))

In [44]:
data_dict.keys()

dict_keys(['symbols', 'year', 'Total Revenue', 'Gross Profit', 'Interest Expense (Income) - Net Operating', 'Operating Income', 'Net Income Before Taxes', 'Provision for Income Taxes', 'Minority Interest', 'Net Income', 'Diluted Weighted Average Shares', 'Total Current Assets', 'Cash and Short Term Investments', 'Cash & Equivalents', 'Total Assets', 'Total Current Liabilities', 'Current Port. of LT Debt/Capital Leases', 'Total Liabilities', 'Total Long Term Debt', 'Total Equity', "Total Liabilities & Shareholders' Equity", 'Cash From Operating Activities', 'Free Cash Flow', 'Cash & Due from Banks'])

In [41]:
columns_rename = {
    "net_operating_cash_flow",
    "total_assets",
    "total_liabilities",
    "total_current_liabilities",
    "total_equity",
    "Total Revenue" : "total_revenue",
    "Net Income Before Taxes" : "net_income",
    "total_debt",
    "stockholders_equity",
    "ebit",
    "ebitda",
    "cash_and_short_term_investments",
    "cash_only",
    "total_cash_and_due_from_banks",
    "diluted_shares_outstanding",
    "Gross Profit" : "gross_income",
    "pretax_income",
    "income_taxes",
    "total_non_current_assets",
    "free_cash_flow",
    "Interest Expense (Income) - Net Operating" : "interest_expense_non_operating",
    "Operating Income" : "operating_income"
}

In [9]:
dc = pd.concat(data_list)
dc

,symbols,year,Total Revenue,Gross Profit,Interest Expense (Income) - Net Operating,Operating Income,Net Income Before Taxes,Provision for Income Taxes,Minority Interest,Net Income,...,Total Assets,Total Current Liabilities,Current Port. of LT Debt/Capital Leases,Total Liabilities,Total Long Term Debt,Total Equity,Total Liabilities & Shareholders' Equity,Cash From Operating Activities,Free Cash Flow,Cash & Due from Banks
0,CBRE.JK,2024-03-31,1.509828e+10,7.803310e+09,-3.555720e+09,5.681560e+09,2.125850e+09,1.811800e+08,NaN,1.944670e+09,...,3.581541e+11,3.519424e+10,2.277416e+10,1.875265e+11,1.476099e+11,1.706276e+11,3.581541e+11,8.843000e+08,-1.731520e+11,NaN
1,CBRE.JK,2023-12-31,9.207690e+09,-3.009170e+09,-4.422300e+08,-5.824970e+09,-7.129790e+09,1.104900e+08,NaN,-7.240290e+09,...,3.529656e+11,3.004078e+10,1.836360e+10,1.842827e+11,1.540195e+11,1.686829e+11,3.529656e+11,-5.963440e+09,-2.027244e+10,NaN
2,CBRE.JK,2023-09-30,9.790480e+09,4.455230e+09,-4.384000e+08,1.808810e+09,1.402250e+09,1.174900e+08,NaN,1.284770e+09,...,2.086194e+11,1.252756e+10,4.811270e+09,3.259236e+10,1.994648e+10,1.760270e+11,2.086194e+11,2.140570e+09,-2.634060e+09,NaN
3,CBRE.JK,2023-06-30,1.099500e+10,5.803210e+09,-1.009160e+09,2.636220e+09,2.725280e+09,1.319400e+08,NaN,2.593340e+09,...,2.081230e+11,1.209779e+10,4.729580e+09,3.338078e+10,2.118036e+10,1.747422e+11,2.081230e+11,6.114250e+09,-6.877900e+08,NaN
0,LAJU.JK,2024-03-31,4.310198e+10,8.153470e+09,-1.080130e+09,4.015200e+09,3.251980e+09,6.491100e+08,NaN,2.602870e+09,...,1.907945e+11,3.217262e+10,1.010453e+10,5.920707e+10,2.595310e+10,1.315874e+11,1.907945e+11,2.736320e+09,-1.155389e+10,NaN
1,LAJU.JK,2023-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,LAJU.JK,2023-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,LAJU.JK,2023-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,ATAP.JK,2024-03-31,5.546420e+09,2.408570e+09,-8.139700e+08,1.006790e+09,2.081500e+08,7.791000e+07,NaN,1.302300e+08,...,1.128547e+11,1.918887e+10,NaN,3.313417e+10,1.362790e+10,7.972054e+10,1.128547e+11,-1.919800e+08,-3.691700e+08,NaN
1,ATAP.JK,2023-12-31,3.921060e+09,1.649910e+09,NaN,-2.165500e+08,-3.770000e+06,1.718700e+08,NaN,-1.756400e+08,...,1.137727e+11,1.927705e+10,NaN,3.418235e+10,1.463725e+10,7.959031e+10,1.137727e+11,-2.042590e+09,-2.179400e+09,NaN


In [14]:
df = pd.concat(data_list)
df

,symbols,year,Total Revenue,Gross Profit,Interest Expense (Income) - Net Operating,Operating Income,Net Income Before Taxes,Provision for Income Taxes,Minority Interest,Net Income,...,Total Assets,Total Current Liabilities,Current Port. of LT Debt/Capital Leases,Total Liabilities,Total Long Term Debt,Total Equity,Total Liabilities & Shareholders' Equity,Cash From Operating Activities,Free Cash Flow,Cash & Due from Banks
0,CBRE.JK,2024-03-31,15098.28,7803.31,-3555.72,5681.56,2125.85,181.18,NaN,1944.67,...,358154.11,35194.24,22774.16,187526.54,147609.89,170627.56,358154.11,884.3,-173152,NaN
1,CBRE.JK,2023-12-31,9207.69,-3009.17,-442.23,-5824.97,-7129.79,110.49,NaN,-7240.29,...,352965.59,30040.78,18363.6,184282.7,154019.51,168682.89,352965.59,-5963.44,-20272.44,NaN
2,CBRE.JK,2023-09-30,9790.48,4455.23,-438.4,1808.81,1402.25,117.49,NaN,1284.77,...,208619.37,12527.56,4811.27,32592.36,19946.48,176027.01,208619.37,2140.57,-2634.06,NaN
3,CBRE.JK,2023-06-30,10995,5803.21,-1009.16,2636.22,2725.28,131.94,NaN,2593.34,...,208123.02,12097.79,4729.58,33380.78,21180.36,174742.25,208123.02,6114.25,-687.79,NaN
0,LAJU.JK,2024-03-31,43101.98,8153.47,-1080.13,4015.2,3251.98,649.11,NaN,2602.87,...,190794.5,32172.62,10104.53,59207.07,25953.1,131587.44,190794.5,2736.32,-11553.89,NaN
1,LAJU.JK,2023-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,LAJU.JK,2023-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,LAJU.JK,2023-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,ATAP.JK,2024-03-31,5546.42,2408.57,-813.97,1006.79,208.15,77.91,NaN,130.23,...,112854.72,19188.87,NaN,33134.17,13627.9,79720.54,112854.72,-191.98,-369.17,NaN
1,ATAP.JK,2023-12-31,3921.06,1649.91,NaN,-216.55,-3.77,171.87,NaN,-175.64,...,113772.66,19277.05,NaN,34182.35,14637.25,79590.31,113772.66,-2042.59,-2179.4,NaN


In [12]:
df[df["symbols"] == "CBRE.JK"]

,symbols,year,Total Revenue,Gross Profit,Interest Expense (Income) - Net Operating,Operating Income,Net Income Before Taxes,Provision for Income Taxes,Minority Interest,Net Income,...,Total Assets,Total Current Liabilities,Current Port. of LT Debt/Capital Leases,Total Liabilities,Total Long Term Debt,Total Equity,Total Liabilities & Shareholders' Equity,Cash From Operating Activities,Free Cash Flow,Cash & Due from Banks
0,CBRE.JK,2024-03-31,15098280000.0,7803310000.0,-3555720000.0,5681560000.0,2125850000.0,181180000.0,NaN,1944670000.0,...,358154110000.0,35194240000.0,22774160000.0,187526540000.0,147609890000.0,170627560000.0,358154110000.0,884300000.0,-173152000000.0,NaN
1,CBRE.JK,2023-12-31,9207690000.0,-3009170000.0,-442230000.0,-5824970000.0,-7129790000.0,110490000.0,NaN,-7240290000.0,...,352965590000.0,30040780000.0,18363600000.0,184282700000.0,154019510000.0,168682890000.0,352965590000.0,-5963440000.0,-20272440000.0,NaN
2,CBRE.JK,2023-09-30,9790480000.0,4455230000.0,-438400000.0,1808810000.0,1402250000.0,117490000.0,NaN,1284770000.0,...,208619370000.0,12527560000.0,4811270000.0,32592360000.0,19946480000.0,176027010000.0,208619370000.0,2140570000.0,-2634060000.0,NaN
3,CBRE.JK,2023-06-30,10995000000.0,5803210000.0,-1009160000.0,2636220000.0,2725280000.0,131940000.0,NaN,2593340000.0,...,208123020000.0,12097790000.0,4729580000.0,33380780000.0,21180360000.0,174742250000.0,208123020000.0,6114250000.0,-687790000.0,NaN


In [11]:
df[['income_taxes', 'interest_expense_non_operating']] *= -1

{'USD': {'IDR': 16248.19}, 'datetime': '2024-06-03 18:15:39'}